In [1]:
import pandas as pd
import geopandas as gpd
import folium
import shapely
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import font_manager, rc, rcParams
def set_korea_font():
    font_name = font_manager.FontProperties(fname="/System/Library/Fonts/Supplemental/AppleGothic.ttf").get_name()
    rc('font', family=font_name)
    rcParams.update({'font.size': 11})
    rcParams['axes.unicode_minus'] = False  
set_korea_font()

In [2]:
child = pd.read_csv("../data/original/9.오산시_어린이보호구역(re).csv")

In [3]:
child.head()

,시설종류,시설명,CCTV설치여부,CCTV설치대수,보호구역도로폭,보호구역_경도,보호구역_위도
0,특수학교,성심학교,N,0,25,127.017521,37.176762
1,초등학교,가수초등학교,Y,1,12,127.047957,37.148103
2,초등학교,광성초등학교,Y,2,12,127.039709,37.184197
3,초등학교,대호초등학교,Y,3,12,127.057889,37.159106
4,초등학교,매홀초등학교,Y,2,6,127.063282,37.162749


In [4]:
child["CCTV설치여부"].value_counts()

Y    75
N    16
Name: CCTV설치여부, dtype: int64

In [5]:
child[child["CCTV설치여부"] == "N"]

,시설종류,시설명,CCTV설치여부,CCTV설치대수,보호구역도로폭,보호구역_경도,보호구역_위도
0,특수학교,성심학교,N,0,25,127.017521,37.176762
30,유치원,오산유치원,N,0,3,127.069549,37.152703
31,유치원,오산대유치원,N,0,6,127.063207,37.153567
61,어린이집,베이비캐슬어린이집,N,0,4,127.075205,37.162708
63,어린이집,동부어린이집,N,0,4,127.075311,37.163121
65,어린이집,둥근해어린이집,N,0,7,127.056508,37.161239
67,어린이집,경희어린이집,N,0,6,127.051706,37.145600
68,어린이집,그린어린이집,N,0,4,127.069611,37.149776
70,어린이집,리틀램어린이집,N,0,3,127.035169,37.142546
72,어린이집,중앙어린이집,N,0,13,127.063702,37.153427


In [6]:
def cctv(ans):
    if ans == "Y":
        return "red"
    else:
        return "blue"

In [7]:
m = folium.Map(location = [37.16501470733718, 127.05422687735454], tiles = 'openstreetmap', zoom_start = 12)

for i in range(0, len(child)):
    folium.Circle(
        location = [child.iloc[i]["보호구역_위도"], child.iloc[i]["보호구역_경도"]],
        radius = 300,
        color = cctv(child.iloc[i]["CCTV설치여부"]),
    ).add_to(m)

In [8]:
m

같이 스쿨존끼리 붙어있는 곳이 많다

아니면 아예 시골쪽

스쿨존이 반경 500m라서 저렇게 그려보았다

## 추가 (새로 만들어진 함수 사용)

### 과속단속 카메라 유무 새로 구하기

In [9]:
from geopy.distance import geodesic

In [10]:
overspeed_cam = pd.read_csv("../Feature/data/overspeed_cam.csv")
overspeed_cam.head()

,lat,lon
0,37.150892,127.090748
1,37.150892,127.090748
2,37.148336,127.083997
3,37.159936,127.057906
4,37.149489,127.065567


In [11]:
overspeed_cam['geometry'] = list(zip(overspeed_cam['lat'], overspeed_cam['lon']))

In [12]:
def overspeedcam_count(lat, lon, radi, refer_data, idx=False):
  """
  input: origin lat lon, radi(m)
  output: following point count
  optional: (idx of point, count)
  """

  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)
  if idx:
    return refer_data[bool_mask].index, sum(bool_mask)
  else:
    return sum(bool_mask)

In [13]:
child['overspeed'] = np.zeros(len(child))
for i in range(len(child)):
    child.overspeed[i] = overspeedcam_count(
        child.iloc[i]["보호구역_위도"], child.iloc[i]["보호구역_경도"], 300, overspeed_cam)

/Users/jominju/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [14]:
child.overspeed.value_counts()

0.0    60
1.0    20
2.0    10
3.0     1
Name: overspeed, dtype: int64

과속단속 카메라가 없는곳이 60개... 잘 겹쳐서 설치해야할 듯

In [15]:
child["isCam"] = child["overspeed"] > 0

In [16]:
child.isCam.value_counts()

False    60
True     31
Name: isCam, dtype: int64

In [17]:
def over_cctv(ans):
    if ans == True:
        return "red"
    else:
        return "blue"

In [29]:
m = folium.Map(location = [37.16501470733718, 127.05422687735454], tiles = 'openstreetmap', zoom_start = 12)

for i in range(0, len(child)):
    folium.Circle(
        location = [child.iloc[i]["보호구역_위도"], child.iloc[i]["보호구역_경도"]],
        radius = 300,
        color = over_cctv(child.iloc[i]["isCam"]),
    ).add_to(m)
    
for i in range(0, len(overspeed_cam)):
    folium.Circle(
        location = [overspeed_cam.iloc[i]["lat"], overspeed_cam.iloc[i]["lon"]],
        radius = 10,
        color = "black"
    ).add_to(m)

In [30]:
m

스쿨존이 겹쳐져 있는 부분은 중심지에 하나를 놓으면 둘 다 커버가 되지만


스쿨존이 떨어져 있으면 무조건 그 스쿨존에는 하나를 설치해야한다

In [20]:
accident = pd.read_csv("../data/crawling/GIS/2021_01_12_03_44_final.csv")

In [21]:
accident.head()

,Unnamed: 0,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,...,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,lat,lon
0,0,2017012800100166,2017년 1월 28일 14시,토요일,경기도 오산시 금암동,경상사고,0,0,6,0,...,승용,남,62세,상해없음,승용,남,39세,경상,37.174346,127.050546
1,1,2017030600100321,2017년 3월 6일 15시,월요일,경기도 오산시 금암동,부상신고사고,0,0,0,1,...,승합,남,54세,상해없음,보행자,남,9세,부상신고,37.175516,127.049976
2,2,2017030600100420,2017년 3월 6일 18시,월요일,경기도 오산시 원동,중상사고,0,1,0,0,...,승합,남,48세,상해없음,보행자,여,12세,중상,37.136190,127.082544
3,3,2017031500100415,2017년 3월 15일 17시,수요일,경기도 오산시 오산동,중상사고,0,1,0,0,...,승합,여,53세,상해없음,자전거,남,11세,중상,37.153618,127.077133
4,4,2017032100100170,2017년 3월 21일 09시,화요일,경기도 오산시 세교동,중상사고,0,1,1,0,...,건설기계,남,54세,상해없음,승용,여,31세,중상,37.186427,127.038726


In [22]:
accident.shape

(691, 25)

In [23]:
school = pd.read_csv("../data/original/9.오산시_어린이보호구역(re).csv", encoding = "utf-8")

def isSchoolZone(lat, lon, radi, refer_data): # radi = 500 이지만 순서 통일을 위해 default값 지정 x
    data_loc = pd.Series(list(zip(refer_data["보호구역_위도"], refer_data["보호구역_경도"])))
    count_sum = data_loc.apply(lambda x: geodesic((lat, lon),x).m <= 300).sum()
    return count_sum > 0

In [24]:
accident["schoolZone"] = True

for i in range(len(accident)):
    accident["schoolZone"][i] = isSchoolZone(
        accident.iloc[i]["lat"], accident.iloc[i]["lon"], 0, school)

/Users/jominju/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [25]:
accident.head()

,Unnamed: 0,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,...,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,lat,lon,schoolZone
0,0,2017012800100166,2017년 1월 28일 14시,토요일,경기도 오산시 금암동,경상사고,0,0,6,0,...,남,62세,상해없음,승용,남,39세,경상,37.174346,127.050546,True
1,1,2017030600100321,2017년 3월 6일 15시,월요일,경기도 오산시 금암동,부상신고사고,0,0,0,1,...,남,54세,상해없음,보행자,남,9세,부상신고,37.175516,127.049976,False
2,2,2017030600100420,2017년 3월 6일 18시,월요일,경기도 오산시 원동,중상사고,0,1,0,0,...,남,48세,상해없음,보행자,여,12세,중상,37.136190,127.082544,True
3,3,2017031500100415,2017년 3월 15일 17시,수요일,경기도 오산시 오산동,중상사고,0,1,0,0,...,여,53세,상해없음,자전거,남,11세,중상,37.153618,127.077133,True
4,4,2017032100100170,2017년 3월 21일 09시,화요일,경기도 오산시 세교동,중상사고,0,1,1,0,...,남,54세,상해없음,승용,여,31세,중상,37.186427,127.038726,True


In [26]:
accident.schoolZone.value_counts()

True     468
False    223
Name: schoolZone, dtype: int64

도대체가 알수가없다

In [27]:
data = pd.read_csv("../data/crawling/school_zone/경기도_오산시_어린이보호구역_20200916_1605839121035_14173.csv"
                  , encoding = "cp949")

In [28]:
data.head()

,시설종류,대상시설명,소재지도로명주소,소재지지번주소,위도,경도,관리기관명,관할경찰서명,CCTV설치여부,CCTV설치대수,보호구역도로폭,데이터기준일자
0,특수학교,성심학교,경기도 오산시 독산성로 140-24 (지곶동),경기도 오산시 지곶동 559-1,37.176762,127.017521,경기도 오산시,오산경찰서,N,0,25,2020-09-16
1,초등학교,가수초등학교,경기도 오산시 가수로 50 (가수동),경기도 오산시 가수동 10-1,37.148103,127.047957,경기도 오산시,오산경찰서,Y,1,12,2020-09-16
2,초등학교,광성초등학교,경기도 오산시 수목원로 577-18 (세교동),경기도 오산시 세교동 623,37.184197,127.039709,경기도 오산시,오산경찰서,Y,2,12,2020-09-16
3,초등학교,대호초등학교,경기도 오산시 궐리사로46번길 3 (궐동),경기도 오산시 궐동 661,37.159106,127.057889,경기도 오산시,오산경찰서,Y,3,12,2020-09-16
4,초등학교,매홀초등학교,경기도 오산시 청학로147번길 10 (수청동),경기도 오산시 수청동 534-1,37.162749,127.063282,경기도 오산시,오산경찰서,Y,2,6,2020-09-16
